导入包

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import shap
from tqdm import tqdm
import os
import time
from collections import defaultdict
import textwrap

class Config:
    """
    Configuration class for all adjustable parameters.
    """
    # File paths configuration
    INPUT_FILE = 'Original features.xlsx'
    OUTPUT_DIR = 'feature_engineering_output'

    # Core algorithm parameters
    RANDOM_STATE = 0
    PEARSON_CORR_THRESHOLD = 0.80

    # Random Forest Regressor hyperparameters
    RF_N_ESTIMATORS = 2000
    RF_MIN_SAMPLES_LEAF = 1
    RF_MIN_SAMPLES_SPLIT = 2
    RF_MAX_FEATURES = 'sqrt'
    RF_MAX_DEPTH = None
    RF_N_JOBS = -1

    # SHAP value calculation and Cross-validation parameters
    PERFORMANCE_METRIC = 'mae' # Options: 'mae' (lower is better) or 'r2' (higher is better)
    SHAP_COARSE_SELECTION_PERCENT = 1 # Percentage of features to keep during SHAP coarse selection (between 0 and 1)

def load_data_from_excel(config: Config):
    """
    Loads data from an Excel file, extracts features (X) and target (Y) by column position,
    and performs data cleaning.

    Args:
        config (Config): Configuration object containing input file path and other settings.

    Returns:
        tuple: A tuple containing:
            - X_cleaned (pd.DataFrame): Cleaned feature data.
            - y_cleaned (pd.Series): Cleaned target data.
            - df_original (pd.DataFrame): A copy of the original DataFrame, with rows containing missing values removed.
    """
    print(f"Loading data from '{config.INPUT_FILE}'...")
    if not os.path.exists(config.INPUT_FILE):
        print(f"Error: File '{config.INPUT_FILE}' does not exist.")
        exit()

    df = pd.read_excel(config.INPUT_FILE)
    df_original = df.copy() # Keep a copy of the original DataFrame for final output

    try:
        all_col_names = df.columns
        # Feature columns: from the second column (index 1) to the third-to-last column (index -2)
        feature_col_names = all_col_names[1:-2]
        
        # Target column: the second-to-last column (index -2)
        # Note: Adjust this index based on your specific target column.
        # For example:
        # If 'band alignment' is the second-to-last column: target_col_name = all_col_names[-2]
        # If 'shift range' is the very last column: target_col_name = all_col_names[-1]
        target_col_name = all_col_names[-1]
        
        # Dynamically describe the extracted feature column range
        if len(feature_col_names) > 1:
            feature_range_str = f"from column '{feature_col_names[0]}' to '{feature_col_names[-1]}'"
        elif len(feature_col_names) == 1:
            feature_range_str = f"from column '{feature_col_names[0]}'"
        else:
            feature_range_str = "with no features" # Case where feature_col_names is empty

        print(f"Extracting features {feature_range_str} and target '{target_col_name}'.")

        if target_col_name in feature_col_names:
            print(f"Critical Error: Target column '{target_col_name}' is also identified as a feature column. Please check Excel column order or slicing rules.")
            exit()

        X = df[feature_col_names]
        Y = df[target_col_name]

    except IndexError:
        print("Error: Column index out of bounds. Ensure the Excel file has at least 3 columns (e.g., SampleID, Feature..., Target, Ignored).")
        exit()
    except Exception as e: # Catch other potential errors during column access, e.g., empty feature_col_names
        print(f"Error during column extraction: {e}. Please check your Excel file structure and the slicing logic.")
        exit()
    
    print(f"Extracted features X, shape: {X.shape}")
    print(f"Extracted target Y '{Y.name}', shape: {Y.shape}")
    
    # Combine X and Y for missing value handling
    combined_df = pd.concat([X, Y], axis=1)
    # Attempt to convert all columns to numeric, coercing errors to NaN
    for col in combined_df.columns:
        combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')
    
    original_rows = len(combined_df)
    # Drop rows containing any NaN values
    combined_df_cleaned = combined_df.dropna()
    
    if len(combined_df_cleaned) < original_rows:
        print(f"Warning: {original_rows - len(combined_df_cleaned)} rows with missing values have been removed.")
        # Update the original DataFrame copy to match the cleaned data rows
        df_original = df_original.loc[combined_df_cleaned.index]

    # Separate X and Y again from the cleaned combined data
    X_cleaned = combined_df_cleaned.drop(columns=[Y.name])
    y_cleaned = combined_df_cleaned[Y.name]

    print(f"Data loading complete. X shape: {X_cleaned.shape}, y shape: {y_cleaned.shape}")
    return X_cleaned, y_cleaned, df_original

def step1_filter_high_correlated_features(X, y, config: Config):
    """
    Stage 1: Filter method to remove highly correlated features based on Pearson correlation coefficient.
    For any pair of features, if their Pearson correlation exceeds the threshold, the feature with
    higher correlation to the target variable is kept, and the other is removed.

    Args:
        X (pd.DataFrame): Input feature data.
        y (pd.Series): Target variable data.
        config (Config): Configuration object containing correlation threshold and other settings.

    Returns:
        tuple: A tuple containing:
            - X_filtered (pd.DataFrame): Filtered feature data.
            - to_drop_list (list): List of names of features that were removed.
    """
    print("\n--- Stage 1: Filtering Highly Correlated Features ---")
    
    if X.shape[1] <= 1:
        print("Insufficient number of features (<=1) for Pearson correlation filtering.")
        return X.copy(), []
    if X.shape[0] <= 1:
        print("Insufficient number of samples (<=1) for Pearson correlation filtering.")
        return X.copy(), []

    # Calculate the absolute Pearson correlation matrix between features
    corr_matrix = X.corr(method='pearson').abs()
    
    # Calculate the absolute Pearson correlation coefficient for each feature with the target variable
    feature_y_corr = {}
    for col in X.columns:
        corr_val = X[col].corr(y)
        feature_y_corr[col] = abs(corr_val) if not pd.isna(corr_val) else -1.0 # NaN values are set to -1 to ensure they are sorted last

    # Sort features in descending order based on their correlation with the target variable
    sorted_features = sorted(X.columns, key=lambda col: feature_y_corr[col], reverse=True)

    kept_features_final = [] # List of features finally kept
    all_dropped_features_set = set() # Set of all features marked for dropping
    retained_to_dropped_map = defaultdict(list) # Maps retained features to features they caused to be dropped

    # Iterate through sorted features to perform high correlation feature removal
    for current_feature in sorted_features:
        # If the current feature has already been marked for dropping, skip it
        if current_feature in all_dropped_features_set:
            continue

        # Add the current feature to the kept list
        kept_features_final.append(current_feature)

        # Iterate through other features to check correlation with the current feature
        for other_feature in sorted_features:
            # Skip self-comparison or features already marked for dropping
            if other_feature == current_feature or other_feature in all_dropped_features_set:
                continue

            # Get the correlation coefficient between the feature pair
            pair_corr = corr_matrix.loc[current_feature, other_feature]
            # If the correlation coefficient exceeds the threshold, mark the 'other feature' for dropping
            if pair_corr > config.PEARSON_CORR_THRESHOLD:
                all_dropped_features_set.add(other_feature)
                retained_to_dropped_map[current_feature].append(other_feature)
    
    to_drop_list = list(all_dropped_features_set)

    if to_drop_list:
        print(f"The following features are removed based on Pearson correlation (threshold>{config.PEARSON_CORR_THRESHOLD}):")
        
        # Prepare structured data for printing
        output_data = []
        for kept_feat in kept_features_final:
            if kept_feat in retained_to_dropped_map:
                dropped_feats_for_this_kept = retained_to_dropped_map[kept_feat]
                output_data.append({
                    'Kept Feature': kept_feat,
                    'Dropped Features_list': dropped_feats_for_this_kept
                })
        
        if output_data:
            # Calculate the maximum width for the first column (kept features), including the header
            max_kept_len = max(len(row['Kept Feature']) for row in output_data)
            col1_header = 'Kept Feature'
            col1_width = max(max_kept_len, len(col1_header)) + 2 # Add 2 extra spaces for column padding

            col2_header = 'Dropped Features'
            terminal_width = 100 # Terminal display width, adjustable as needed

            # Calculate the absolute starting position for the second column's content (characters from line start)
            target_col2_start_pos = col1_width + 1 # First column width + 1 space for padding
            
            # textwrap.fill will add a prefix indent string to all lines
            indent_str_for_textwrap = ' ' * target_col2_start_pos

            # Print table header
            print(f"{col1_header:<{col1_width}} {col2_header:<}")
            print(f"{'-' * col1_width} {'-' * len(col2_header)}")

            # Print each row of data
            for row in output_data:
                kept_feat = row['Kept Feature']
                original_dropped_feats_list = sorted(row['Dropped Features_list'])

                # Step 1: Replace spaces within feature names with a placeholder and add quotes for textwrap to handle correctly
                safe_dropped_feats_list = []
                for d in original_dropped_feats_list:
                    safe_d = d.replace(' ', '___FEAT_SPACE___') # Use a unique placeholder
                    safe_dropped_feats_list.append(f"'{safe_d}'")
                
                # Step 2: Join the processed feature name list with commas and spaces
                dropped_feats_str_temp = ", ".join(safe_dropped_feats_list)

                # Step 3: Use textwrap.fill to wrap the text, enabling automatic line breaks and indentation
                wrapped_lines_with_indent = textwrap.fill(
                    dropped_feats_str_temp, 
                    width=terminal_width, 
                    initial_indent=indent_str_for_textwrap, 
                    subsequent_indent=indent_str_for_textwrap, 
                    break_long_words=False # Try not to break words
                ).split('\n')

                # Step 4: Replace placeholders back with spaces to restore original feature names
                wrapped_dropped_lines_clean = [line.replace('___FEAT_SPACE___', ' ') for line in wrapped_lines_with_indent]

                # Print the first line: Kept Feature + first line of dropped features content
                # The dropped features content needs to remove the prefix indent added by textwrap,
                # as the f-string already handles the first column's width and spacing.
                print(f"{kept_feat:<{col1_width}} {wrapped_dropped_lines_clean[0][len(indent_str_for_textwrap):]}")

                # Print subsequent lines (if dropped features content spans multiple lines)
                for i in range(1, len(wrapped_dropped_lines_clean)):
                    # For subsequent lines, the first column is empty but needs to be filled to the same width
                    # as the first column to maintain alignment.
                    # The dropped features content needs to remove the prefix indent added by textwrap
                    # and ensure printing starts from the correct position.
                    print(f"{'':<{col1_width}}{wrapped_dropped_lines_clean[i][col1_width:]}") 
            
        else:
            print("  (No feature pairs exceeded the Pearson correlation threshold, no features removed.)")
        
        X_filtered = X.drop(columns=to_drop_list)
    else:
        print("No feature pairs exceeded the Pearson correlation threshold, no features removed.")
        X_filtered = X.copy()
    
    print(f"Number of features after filtering: {X_filtered.shape[1]}")
    return X_filtered, to_drop_list

def step2_embed_shap_coarse_selection(X_filtered, y, config: Config):
    """
    Stage 2: Embedded method for coarse feature selection based on SHAP values.
    Uses Leave-One-Out Cross-Validation (LOOCV) to calculate the average absolute SHAP value for each feature,
    keeping the top N% most important features.

    Args:
        X_filtered (pd.DataFrame): Feature data after Stage 1 filtering.
        y (pd.Series): Target variable data.
        config (Config): Configuration object containing Random Forest parameters and SHAP coarse selection percentage.

    Returns:
        tuple: A tuple containing:
            - X_shap_coarse (pd.DataFrame): Feature data after SHAP coarse selection.
            - features_to_drop (list): List of names of features that were removed.
    """
    print("\n--- Stage 2: SHAP Coarse Feature Selection ---")
    
    if X_filtered.shape[1] == 0:
        print("No features available for SHAP coarse selection.")
        return X_filtered.copy(), []
    if X_filtered.shape[0] <= 1:
        print("Insufficient number of samples (<=1) for SHAP coarse selection.")
        return X_filtered.copy(), []

    loo = LeaveOneOut()
    avg_abs_shap_values_per_fold = [] # Stores average absolute SHAP values for each LOOCV fold
    
    n_splits = X_filtered.shape[0]
    print(f"Calculating SHAP values (using Leave-One-Out Cross-Validation, {n_splits} folds)...")
    
    # Check if the training set sample size meets the minimum requirements for RandomForest
    min_samples_for_rf = max(config.RF_MIN_SAMPLES_LEAF, config.RF_MIN_SAMPLES_SPLIT)
    if (n_splits - 1) < min_samples_for_rf:
        print(f"Warning: Training set sample size ({n_splits-1}) is less than the minimum required samples ({min_samples_for_rf}) for RandomForestRegressor. SHAP coarse selection will be skipped.")
        return X_filtered.copy(), []

    # Iterate through LOOCV folds, train the model, and calculate SHAP values
    for train_idx, val_idx in tqdm(loo.split(X_filtered), total=n_splits, desc="SHAP Coarse Selection Progress"):
        X_fold_train, y_fold_train = X_filtered.iloc[train_idx], y.iloc[train_idx]
        
        # Standardize the training data
        scaler = StandardScaler().fit(X_fold_train)
        X_fold_train_scaled = scaler.transform(X_fold_train)
        
        # Initialize and train the RandomForestRegressor
        rf_model = RandomForestRegressor(n_estimators=config.RF_N_ESTIMATORS, max_features=config.RF_MAX_FEATURES, min_samples_leaf=config.RF_MIN_SAMPLES_LEAF, min_samples_split=config.RF_MIN_SAMPLES_SPLIT, random_state=config.RANDOM_STATE, n_jobs=config.RF_N_JOBS)
        
        try:
            rf_model.fit(X_fold_train_scaled, y_fold_train)
            # Use TreeExplainer to calculate SHAP values
            explainer = shap.TreeExplainer(rf_model)
            shap_values_fold = explainer.shap_values(X_fold_train_scaled)
            
            # Handle SHAP value dimensions (typically 1D for regression, multi-dimensional for classification)
            if shap_values_fold.ndim == 1:
                avg_abs_shap_values_per_fold.append(np.abs(shap_values_fold))
            else:
                avg_abs_shap_values_per_fold.append(np.abs(shap_values_fold).mean(axis=0))
        except Exception as e:
            print(f"\nWarning: RandomForestRegressor training or SHAP calculation failed during SHAP coarse selection (possibly due to data issues or improper parameters): {e}")
            continue
    
    if not avg_abs_shap_values_per_fold:
        print("Warning: Failed to calculate any SHAP values, possibly due to small sample size or model training failure. Skipping SHAP coarse selection.")
        return X_filtered.copy(), []

    # Calculate the average absolute SHAP values across all folds and sort by importance in descending order
    feature_importances_shap = pd.Series(np.mean(avg_abs_shap_values_per_fold, axis=0), index=X_filtered.columns).sort_values(ascending=False)
    print("\nFeature Importance (based on average absolute SHAP values from LOOCV):")
    print(feature_importances_shap)

    # Determine the number of features to keep based on the configured percentage
    num_features_to_keep = int(len(feature_importances_shap) * config.SHAP_COARSE_SELECTION_PERCENT)
    # Ensure at least one feature is kept (if available)
    if num_features_to_keep == 0 and len(feature_importances_shap) > 0: num_features_to_keep = 1
    # Ensure the number of features to keep does not exceed the total number of features
    if num_features_to_keep > len(feature_importances_shap): num_features_to_keep = len(feature_importances_shap)
        
    # Get the list of features to keep and features to drop
    features_to_keep = feature_importances_shap.index[:num_features_to_keep].tolist()
    features_to_drop = list(set(X_filtered.columns) - set(features_to_keep))

    if features_to_drop:
        print(f"\nBased on SHAP coarse selection (keeping top {config.SHAP_COARSE_SELECTION_PERCENT*100:.1f}%), the following features are removed: {features_to_drop}")
        X_shap_coarse = X_filtered[features_to_keep]
    else:
        print("\nNo features were removed after SHAP coarse selection.")
        X_shap_coarse = X_filtered.copy()
    
    print(f"Number of features after SHAP coarse selection: {X_shap_coarse.shape[1]}")
    return X_shap_coarse, features_to_drop

def step3_wrapper_shap_iterative_selection(X_shap_coarse, y, config: Config):
    """
    Stage 3: Wrapper method for iterative feature refinement.
    In each iteration, LOOCV is used to evaluate model performance and SHAP feature importance.
    The feature with the lowest SHAP value in the current set is removed until performance no longer
    improves or only one feature remains.

    Args:
        X_shap_coarse (pd.DataFrame): Feature data after Stage 2 SHAP coarse selection.
        y (pd.Series): Target variable data.
        config (Config): Configuration object containing Random Forest parameters, performance metric, etc.

    Returns:
        tuple: A tuple containing:
            - best_features (list): List of names of the finally selected best features.
            - performance_history (list): List of dictionaries, each containing feature count and performance metric for an iteration.
    """
    print("\n--- Stage 3: Iterative Feature Refinement (Leave-One-Out Cross-Validation) ---")
    
    if X_shap_coarse.shape[1] <= 1:
        print("Insufficient number of features (<=1) for iterative refinement.")
        return list(X_shap_coarse.columns), []
    if X_shap_coarse.shape[0] <= 1:
        print("Insufficient number of samples (<=1) for iterative refinement.")
        return list(X_shap_coarse.columns), []

    current_features = list(X_shap_coarse.columns) # List of features used in the current iteration
    best_features = list(current_features) # Records the best feature set found so far
    
    # Set initial best score and comparison function based on the performance metric
    if config.PERFORMANCE_METRIC == 'r2':
        best_score = -np.inf # Higher R2 is better
        is_better = lambda current, best: current > best
    elif config.PERFORMANCE_METRIC == 'mae':
        best_score = np.inf # Lower MAE is better
        is_better = lambda current, best: current < best
    else:
        raise ValueError("Unsupported performance metric. Please choose 'r2' or 'mae'.")

    performance_history = [] # Records performance history for each iteration
    loo = LeaveOneOut()

    # Check if the initial training set sample size meets the minimum requirements for RandomForest
    min_samples_for_rf = max(config.RF_MIN_SAMPLES_LEAF, config.RF_MIN_SAMPLES_SPLIT)
    if (X_shap_coarse.shape[0] - 1) < min_samples_for_rf:
        print(f"Warning: Initial training set sample size ({X_shap_coarse.shape[0]-1}) is less than the minimum required samples ({min_samples_for_rf}) for RandomForestRegressor. Cannot calculate baseline performance, iterative refinement will be skipped.")
        return list(X_shap_coarse.columns), []

    print("Calculating baseline performance for the initial feature set...")
    initial_fold_scores = [] # Stores performance scores for the initial feature set across LOOCV folds
    initial_shap_values = [] # Stores SHAP values for the initial feature set across LOOCV folds (primarily for debugging or future expansion)

    # Iterate through LOOCV folds to evaluate the performance of the initial feature set
    for train_idx, val_idx in tqdm(loo.split(X_shap_coarse), total=X_shap_coarse.shape[0], desc="Baseline Performance Calculation Progress"):
        X_fold_train, y_fold_train = X_shap_coarse.iloc[train_idx][current_features], y.iloc[train_idx]
        X_fold_val, y_fold_val = X_shap_coarse.iloc[val_idx][current_features], y.iloc[val_idx]
        
        # Standardize the training and validation data
        scaler_fold = StandardScaler().fit(X_fold_train)
        X_fold_train_scaled = scaler_fold.transform(X_fold_train)
        X_fold_val_scaled = scaler_fold.transform(X_fold_val)

        # Initialize and train the RandomForestRegressor
        rf_model = RandomForestRegressor(n_estimators=config.RF_N_ESTIMATORS, max_features=config.RF_MAX_FEATURES, min_samples_leaf=config.RF_MIN_SAMPLES_LEAF, min_samples_split=config.RF_MIN_SAMPLES_SPLIT, random_state=config.RANDOM_STATE, n_jobs=config.RF_N_JOBS)
        
        try:
            rf_model.fit(X_fold_train_scaled, y_fold_train)
            y_pred = rf_model.predict(X_fold_val_scaled)
            
            # Calculate performance based on the configured metric
            if config.PERFORMANCE_METRIC == 'r2':
                initial_fold_scores.append(r2_score(y_fold_val, y_pred))
            elif config.PERFORMANCE_METRIC == 'mae':
                initial_fold_scores.append(mean_absolute_error(y_fold_val, y_pred))

            # Calculate SHAP values (though not directly used for baseline performance decision, maintains consistency)
            explainer = shap.TreeExplainer(rf_model)
            shap_values_fold = explainer.shap_values(X_fold_train_scaled)
            if shap_values_fold.ndim == 1:
                initial_shap_values.append(np.abs(shap_values_fold))
            else:
                initial_shap_values.append(np.abs(shap_values_fold).mean(axis=0))
        except Exception as e:
            print(f"\nWarning: RandomForestRegressor training or SHAP calculation failed during baseline performance calculation: {e}")
            continue

    if not initial_fold_scores:
        print("Warning: The initial feature set failed to yield any valid scores, iterative refinement cannot proceed.")
        return list(X_shap_coarse.columns), []

    # Update best score and performance history
    best_score = np.mean(initial_fold_scores)
    performance_history.append({'num_features': len(current_features), config.PERFORMANCE_METRIC: best_score})
    print(f"  Initial feature set ({len(current_features)} features), Average {config.PERFORMANCE_METRIC.upper()}: {best_score:.4f}")
    
    print("Starting iterative feature removal...")
    iteration_count = 0
    # Loop until only one feature remains or performance no longer improves
    while len(current_features) > 1:
        iteration_count += 1
        
        current_iteration_shap_values_for_drop = [] # Stores SHAP values for each fold in the current iteration
        n_splits_current_loop = X_shap_coarse.shape[0]
        
        # Re-check if the training set sample size meets the requirements
        if (n_splits_current_loop - 1) < min_samples_for_rf:
            print(f"  Warning: Iteration {iteration_count} training set sample size ({n_splits_current_loop-1}) is less than the minimum required samples ({min_samples_for_rf}) for RandomForestRegressor. Cannot calculate SHAP values, stopping iteration.")
            break

        # Iterate through LOOCV folds to calculate SHAP values for the current feature set
        for train_idx, val_idx in loo.split(X_shap_coarse):
            X_fold_train, y_fold_train = X_shap_coarse.iloc[train_idx][current_features], y.iloc[train_idx]
            
            scaler_fold = StandardScaler().fit(X_fold_train)
            X_fold_train_scaled = scaler_fold.transform(X_fold_train)
            rf_model = RandomForestRegressor(n_estimators=config.RF_N_ESTIMATORS, max_features=config.RF_MAX_FEATURES, min_samples_leaf=config.RF_MIN_SAMPLES_LEAF, min_samples_split=config.RF_MIN_SAMPLES_SPLIT, random_state=config.RANDOM_STATE, n_jobs=config.RF_N_JOBS)
            try:
                rf_model.fit(X_fold_train_scaled, y_fold_train)
                explainer = shap.TreeExplainer(rf_model)
                shap_values_fold = explainer.shap_values(X_fold_train_scaled)
                if shap_values_fold.ndim == 1:
                    current_iteration_shap_values_for_drop.append(np.abs(shap_values_fold))
                else:
                    current_iteration_shap_values_for_drop.append(np.abs(shap_values_fold).mean(axis=0))
            except Exception as e:
                print(f"  Warning: Iteration {iteration_count} SHAP calculation failed: {e}")
                continue
        
        if not current_iteration_shap_values_for_drop:
            print(f"  Warning: Iteration {iteration_count} failed to calculate SHAP values, cannot remove features. Stopping iteration.")
            break
        
        # Calculate average SHAP values and identify the least important feature
        avg_iteration_shap = np.mean(current_iteration_shap_values_for_drop, axis=0)
        feature_importances_current = pd.Series(avg_iteration_shap, index=current_features).sort_values(ascending=True)
        
        feature_to_drop = feature_importances_current.index[0] # Feature with the lowest SHAP value
        current_features.remove(feature_to_drop) # Remove this feature from the current feature set
        print(f"  Removing least important feature: {feature_to_drop}")

        fold_scores_after_drop = [] # Stores performance scores after feature removal across LOOCV folds
        n_splits_after_drop = X_shap_coarse.shape[0]

        # Re-check if the training set sample size meets the requirements
        if (n_splits_after_drop - 1) < min_samples_for_rf:
            print(f"  Warning: Iteration {iteration_count} training set sample size after feature removal ({n_splits_after_drop-1}) is less than the minimum required samples ({min_samples_for_rf}) for RandomForestRegressor. Cannot evaluate performance, stopping iteration.")
            break

        # Iterate through LOOCV folds to evaluate model performance after feature removal
        for train_idx, val_idx in tqdm(loo.split(X_shap_coarse), total=n_splits_after_drop, desc=f"Evaluating Performance After Removal"):
            X_fold_train, y_fold_train = X_shap_coarse.iloc[train_idx][current_features], y.iloc[train_idx]
            X_fold_val, y_fold_val = X_shap_coarse.iloc[val_idx][current_features], y.iloc[val_idx]
            
            scaler_fold = StandardScaler().fit(X_fold_train)
            X_fold_train_scaled = scaler_fold.transform(X_fold_train)
            X_fold_val_scaled = scaler_fold.transform(X_fold_val)

            rf_model = RandomForestRegressor(n_estimators=config.RF_N_ESTIMATORS, max_features=config.RF_MAX_FEATURES, min_samples_leaf=config.RF_MIN_SAMPLES_LEAF, min_samples_split=config.RF_MIN_SAMPLES_SPLIT, random_state=config.RANDOM_STATE, n_jobs=config.RF_N_JOBS)
            try:
                rf_model.fit(X_fold_train_scaled, y_fold_train)
                y_pred = rf_model.predict(X_fold_val_scaled)
                if config.PERFORMANCE_METRIC == 'r2':
                    fold_scores_after_drop.append(r2_score(y_fold_val, y_pred))
                elif config.PERFORMANCE_METRIC == 'mae':
                    fold_scores_after_drop.append(mean_absolute_error(y_fold_val, y_pred))
            except Exception as e:
                print(f"\nWarning: Iteration {iteration_count} failed to evaluate performance after removal: {e}")
                continue

        if not fold_scores_after_drop:
            print(f"  Warning: Iteration {iteration_count} failed to calculate any valid scores after feature removal, stopping iteration.")
            break

        current_avg_score = np.mean(fold_scores_after_drop)
        performance_history.append({'num_features': len(current_features), config.PERFORMANCE_METRIC: current_avg_score})
        print(f"  Number of features: {len(current_features)}, Average {config.PERFORMANCE_METRIC.upper()}: {current_avg_score:.4f}")
        
        # If current performance is better than historical best, update the best feature set
        if is_better(current_avg_score, best_score):
            best_score, best_features = current_avg_score, list(current_features)
            print(f"    -> Performance improved, current best feature set size: {len(best_features)}")
        else:
            print("    -> Performance did not improve, stopping iteration.")
            break
        
    print(f"\nIterative refinement complete. Best feature set ({len(best_features)} features): {best_features}")
    return best_features, performance_history

if __name__ == "__main__":
    config = Config()
    # Create output directory if it doesn't exist
    os.makedirs(config.OUTPUT_DIR, exist_ok=True)

    # If the input file does not exist, generate dummy data for demonstration
    if not os.path.exists(config.INPUT_FILE):
        print(f"'{config.INPUT_FILE}' not found, creating dummy data...")
        num_samples, feature_cols = 7, 25 
        df_list = [pd.DataFrame([f'Sample_{i+1}' for i in range(num_samples)], columns=['SampleID'])]
        
        # Define some dummy feature names, including names with spaces
        feature_names_from_image = [
            'Chemical potential', 'main group number', 'Overall surface area', 'period number',
            'Vertical IP', 'Electronegativity', 'Electrophilicity index', 'Ionization Energy',
            'Nucleophilicity index', 'S-NM (Å)', 'MendeleevNumber', 'Vertical EA', 'pUnfilled',
            'pValence', 'CovalentRadius', 'Ti-NM (Å)', 'Volume', 'AtomicWeight', 'Number', 'density'
        ]
        # Add generic features to reach the total feature count
        generic_features = [f'F{i+1}' for i in range(feature_cols - len(feature_names_from_image))]
        all_feature_names = generic_features + feature_names_from_image
        
        # Generate dummy feature data
        feature_data = np.random.rand(num_samples, len(all_feature_names))
        df_features = pd.DataFrame(feature_data, columns=all_feature_names)
        df_list.append(df_features)
        
        # Add a dummy column that will be ignored
        df_list.append(pd.DataFrame(np.random.rand(num_samples), columns=['Ignored_G']))
        df_dummy = pd.concat(df_list, axis=1)
        
        # Generate dummy target data and name it 'Shift range (eV)'
        y_values = 5 * df_dummy[all_feature_names[0]] + 3 * df_dummy[all_feature_names[1]]**2 + np.random.randn(num_samples) * 0.5
        df_dummy['Shift range (eV)'] = y_values

        # Ensure column order matches the expectation of load_data_from_excel function (SampleID, Feature..., Ignored_G, Target_Y)
        all_cols_ordered = ['SampleID'] + all_feature_names + ['Ignored_G', 'Shift range (eV)']
        df_dummy = df_dummy[all_cols_ordered]

        df_dummy.to_excel(config.INPUT_FILE, index=False)
        print(f"Dummy data '{config.INPUT_FILE}' created.")

    # Load data
    X_full, y_full, df_original = load_data_from_excel(config)
    
    print("\n--- Outputting Original Feature Correlation Data ---")
    if not X_full.empty:
        # Calculate and save the correlation matrix of original features
        original_corr_matrix = X_full.corr(method='pearson')
        original_corr_path = os.path.join(config.OUTPUT_DIR, f'original_features_correlation_data_{time.strftime("%Y%m%d_%H%M%S")}.xlsx')
        original_corr_matrix.to_excel(original_corr_path, index=True)
        print(f"Saved: Original Feature Correlation Data -> {original_corr_path}")
    else:
        print("No features loaded, cannot calculate original correlation matrix.")

    # Execute Stage 1: Filter highly correlated features
    X_filtered, dropped_pearson_list = step1_filter_high_correlated_features(X_full, y_full, config)

    print("\n--- Outputting Stage 1 Filtered Feature Correlation Data ---")
    if not X_filtered.empty and X_filtered.shape[1] > 1: # Ensure there are at least two features to calculate correlation matrix
        # Calculate and save the correlation matrix of Stage 1 filtered features
        filtered_corr_matrix = X_filtered.corr(method='pearson')
        filtered_corr_path = os.path.join(config.OUTPUT_DIR, f'step1_filtered_features_correlation_data_{time.strftime("%Y%m%d_%H%M%S")}.xlsx')
        filtered_corr_matrix.to_excel(filtered_corr_path, index=True)
        print(f"Saved: Stage 1 Filtered Feature Correlation Data -> {filtered_corr_path}")
    elif X_filtered.shape[1] <= 1:
        print("Insufficient features after Stage 1 filtering (<=1), cannot calculate correlation matrix.")
    else: # X_filtered is empty
        print("No features remaining after Stage 1 filtering, cannot calculate correlation matrix.")

    # Execute Stage 2: SHAP Coarse Feature Selection
    X_shap_coarse, dropped_shap = step2_embed_shap_coarse_selection(X_filtered, y_full, config)
    # Execute Stage 3: Iterative Feature Refinement
    final_features, performance_history = step3_wrapper_shap_iterative_selection(X_shap_coarse, y_full, config)

    print(f"\n--- Outputting Final Results to '{config.OUTPUT_DIR}' Directory ---")

    # Create and save feature engineering summary
    num_dropped_pearson = len(dropped_pearson_list)
    summary_df = pd.DataFrame({
        'Stage': ['Original', 'Stage 1: Pearson Filtering', 'Stage 2: SHAP Coarse Selection', 'Stage 3: SHAP Iterative Selection'],
        'Num_Features': [X_full.shape[1], X_filtered.shape[1], X_shap_coarse.shape[1], len(final_features)],
        'Dropped_Features_Count': [0, num_dropped_pearson, len(dropped_shap), X_shap_coarse.shape[1] - len(final_features)]
    })
    summary_path = os.path.join(config.OUTPUT_DIR, f'feature_engineering_summary_{time.strftime("%Y%m%d_%H%M%S")}.xlsx')
    summary_df.to_excel(summary_path, index=False)
    print(f"Saved: Feature Engineering Summary -> {summary_path}")

    # Save performance history
    history_path = os.path.join(config.OUTPUT_DIR, f'performance_history_{time.strftime("%Y%m%d_%H%M%S")}.xlsx')
    pd.DataFrame(performance_history).to_excel(history_path, index=False)
    print(f"Saved: Performance History -> {history_path}")

    # If the number of final features is greater than 1, calculate and save the correlation matrix of final features
    if len(final_features) > 1:
        final_features_df = df_original[final_features]
        final_corr_matrix = final_features_df.corr(method='pearson')
        final_corr_path = os.path.join(config.OUTPUT_DIR, f'final_features_correlation_data_{time.strftime("%Y%m%d_%H%M%S")}.xlsx')
        final_corr_matrix.to_excel(final_corr_path, index=True)
        print(f"Saved: Final Features Correlation Data -> {final_corr_path}")
    elif len(final_features) == 1:
        print(f"Only one feature remains: {final_features[0]}, cannot calculate correlation matrix.")
    else:
        print("No features remaining, cannot calculate correlation matrix.")
    
    # If final features exist, save the final dataset containing selected features and the target variable
    if final_features:
        cols_to_export = []
        # If the original data contains a 'SampleID' column, export it as well
        if 'SampleID' in df_original.columns:
            cols_to_export.append('SampleID')
        # Add the finally selected feature columns
        cols_to_export.extend(final_features)
        # Add the target variable column
        if y_full.name in df_original.columns:
            cols_to_export.append(y_full.name)
        
        dataset_path = os.path.join(config.OUTPUT_DIR, f'final_engineered_dataset_{time.strftime("%Y%m%d_%H%M%S")}.xlsx')
        df_original[cols_to_export].to_excel(dataset_path, index=False)
        print(f"Saved: Final Engineered Dataset -> {dataset_path}")
    else:
        print("No final features were selected, final dataset not saved.")

    print("\nAll processing complete.")


Loading data from 'Original features.xlsx'...
Extracting features from column 'Number' to 'S-NM (Å)' and target 'Shift range (eV)'.
Extracted features X, shape: (7, 25)
Extracted target Y 'Shift range (eV)', shape: (7,)
Data loading complete. X shape: (7, 25), y shape: (7,)

--- Outputting Original Feature Correlation Data ---
Saved: Original Feature Correlation Data -> feature_engineering_output\original_features_correlation_data_20250710_224045.xlsx

--- Stage 1: Filtering Highly Correlated Features ---
The following features are removed based on Pearson correlation (threshold>0.8):
Kept Feature                  Dropped Features
----------------------------- ----------------
Nucleophilicity index (eV)    'Chemical potential (eV) ',
                              'Electronegativity',
                              'Electrophilicity index (eV) ',
                              'Ionization Energy (eV) ',
                              'S-NM (Å)',
                              'Vertical EA (

SHAP Coarse Selection Progress:  71%|███████▏  | 5/7 [00:08<00:03,  1.70s/it]


KeyboardInterrupt: 